In [1]:
using PrettyTables, Printf, LinearAlgebra, LaTeXStrings
include("LAcodes.jl")

Main.LAcodes

# LU Decomposition

This notebook is designed to show how reordering the computations
of the Gaussian Elimination Algorithm leads to the $A = P L U$ decomposition.

## 1. Theory

We recognized that Gaussian elimination consists of finding suitable matrices $E_i, i=1,2,\dots,k$
that systematically reduce a given matrix $A$ to a row echelon form matrix $U$:

$$
\begin{align}
A x = b & \Leftrightarrow     E_1 A x &=& \;        E_1 b \\
        & \Leftrightarrow E_2 E_1 A x &=& \;    E_2 E_1 b \\
        & & \cdots &\\
        & \Leftrightarrow E_k \dots E_2 E_1 A x &=& \; E_k \dots E_2 E_1 b \\
        & \Leftrightarrow U x &=& \; y
\end{align}
$$

The resulting matrix equation $U x = y$ has $U = E_k \dots E_2 E_1 A$ and $y = E_k \dots E_2 E_1 b$

---
> Given a matrix $A$, we could precompute the matrices $E_i$.
> 
> If we get a set of righthand sides $b$, we then obtain the solution by
>* computing $y = E_k \dots E_2 E_1 b$ followed by
>* solving $U x = y$ by back-substitution
---

Note that the computation of both $U$ and $y$ involves the same product of elementary matrices $E_k \dots E_2 E_1$.

### 1.1 $\ $  A simple Example
Let's consider an example and look at this product:

In [4]:
LAcodes.title("Gaussian Elimination for [A | I ]",sz=20)
A        = [ 1 2 1 0 3; -2 -2 -1 -1 -5; 2 -2 -1 2 6]; M,N = size(A)
identity = Matrix( 1I, M, M)

AI       = [A identity]

p1 = (1,1); E1 = [ 1 0 0; 2 1 0; -2 0 1 ]; A1 = E1 * AI
p2 = (2,2); E2 = [ 1 0 0; 0 1 0;  0 3 1 ]; A2 = E2 * A1;
p3 = (3,4)

# we found
U    = A2[:,1:N]
E2E1 = A2[:,N+1:end]

LAcodes.ge_layout( AI, [E1 A1; E2 A2], (p1,(p1,p2),(p1,p2,p3)), col_divs=size(A,2))

HTML{String}("<div><p style=\"color:blue;font-size:20pt;height:15px;text-align:left;\">Gaussian Elimination for [A | I ]</p></div>")

,,,,1,2,1,0,3,,1,0,0
,,,,-2,-2,-1,-1,-5,,0,1,0
,,,,2,-2,-1,2,6,,0,0,1
1,0,0,,1,2,1,0,3,,1,0,0
2,1,0,,0,2,1,-1,1,,2,1,0
-2,0,1,,0,-6,-3,2,0,,-2,0,1
1,0,0,,1,2,1,0,3,,1,0,0
0,1,0,,0,2,1,-1,1,,2,1,0
0,3,1,,0,0,0,-1,3,,4,3,1


We have kept track of the product of the elementary GE matrices applied with a simple expedient:
just augment the $A$ matrix by $I$!

If we now get a $b = (10 \; 5 \; 11)^t$ for example, we can solve our $A x = b$ system
by computing $y = E_2 E_1 b$

$$
y = \begin{pmatrix} 1&0&0 \\ 2&1&0 \\ 4&3&1 \end{pmatrix} \begin{pmatrix}10 \\ 5 \\ 11 \end{pmatrix} = \begin{pmatrix}10 \\ 25 \\ 66 \end{pmatrix}
$$

followed by the usual backsubstitution problem $U x =y$:

$$
\begin{pmatrix}
 1&  2&  1&   0&  3\\
 0&  2&  1&  -1&  1\\
 0&  0&  0&  -1&  3
\end{pmatrix}   x  = \begin{pmatrix}10 \\ 25 \\ 66 \end{pmatrix}
$$

---
Note that we need to multiply out the $E_i$ matrices to do this. Can we do better?

Look again at our equation
$$y = E_2 E_1 b \Leftrightarrow E_1^{-1} E_2^{-1} y = b$$

The inverses of the $E_i$ matrices are trivial: just change the signs of the off-diagonal terms.<br>
> It turns out that the product is trivial to: think of superposing the matrices and keeping the non-zero entries:<br>
> the $i^{th}$ column of the product is the $i^{th}$ column pf $E_i$

$$ E_1^{-1} E_2^{-1} = \begin{pmatrix}\ 1 & 0 & 0\\ -2 & 1 & 0\\ 2 &  0 & 1 \end{pmatrix}
                       \begin{pmatrix}\ 1 & 0 & 0\\  0 & 1 & 0\\ 0 & -3 & 1 \end{pmatrix}
                     = \begin{pmatrix}\ 1 & 0 & 0\\ -2 & 1 & 0\\ 2 & -3 & 1 \end{pmatrix} = L
$$

> The result is a **unit lower triangular matrix** which we will call $L$: we need to solve $L y = b$.

If we transcribe this matrix equation, we see that it is trivial: the equations are
$$
\begin{align}
y_1 =& 10\\
y_2 =& 5 + 2 y_1\\
y_3 =& 11 -2 y_1 + 3 y_2
\end{align}
$$
**The equations are trivially solvable from the top down by forward substitution!**

---
It turns out this pattern always holds provided that
* we use GE, not GJ
* we do not scale any row while running a GE computation (that is a *choice*)
* we do not interchange any rows while running a GE computation (might be unavoidable)

> In summary: given these two conditions hold, we have found a unit lower triangular matrix $L$ and a row form matrix $U$<br>
such that $$A = L U,$$ which we use to solve
>$$ A x = b \Leftrightarrow L (U x) = b \Leftrightarrow \quad \begin{cases} L y = b\\ U x = y \end{cases}$$

### 1.3. $\ $ An example with a larger matrix

In [46]:
A = [ 1    2   1   0    3;
     -1    0   0  -1   -2;
     -2  -10  -2   2   -6;
     -3  -14  -7   6  -12 ]

LAcodes.title("Add a right hand side to check y",sz=15)
M,N      = size(A)
b        = [-1;1;-2;5]

AI       = [A b]

p1=(1,1); E1 = [ 1 0 0 0; 1 1 0 0;  2 0 1 0; 3 0 0 1]; A1 = E1 * AI
p2=(2,2); E2 = [ 1 0 0 0; 0 1 0 0;  0 3 1 0; 0 4 0 1]; A2 = E2 * A1

LAcodes.ge_layout( AI, [E1 A1; E2 A2], [p1,(p1,p2),(p1,p2,(3,3))], col_divs=size(A,2))

HTML{String}("<div><p style=\"color:blue;font-size:15pt;height:15px;text-align:left;\">Add a right hand side to check y</p></div>")

,,,,,1,2,1,0,3,,-1
,,,,,-1,0,0,-1,-2,,1
,,,,,-2,-10,-2,2,-6,,-2
,,,,,-3,-14,-7,6,-12,,5
1,0,0,0,,1,2,1,0,3,,-1
1,1,0,0,,0,2,1,-1,1,,0
2,0,1,0,,0,-6,0,2,0,,-4
3,0,0,1,,0,-8,-4,6,-3,,2
1,0,0,0,,1,2,1,0,3,,-1
0,1,0,0,,0,2,1,-1,1,,0
0,3,1,0,,0,0,3,-1,3,,-4


In [32]:
LAcodes.title("The L matrix is made up from the inverses of the "*L"E_i",sz=20)
LAcodes.title("don't forget the sign change!", sz=15, color="red")
L = inv(E1)*inv(E2)

println("Using the first column from E_1 and the 2nd column form E2, we see L =")
LAcodes.ge_layout( Int64.(L) )

HTML{String}("<div><p style=\"color:blue;font-size:20pt;height:15px;text-align:left;\">The L matrix is made up from the inverses of the \$E_i\$</p></div>")

HTML{String}("<div><p style=\"color:red;font-size:15pt;height:15px;text-align:left;\">don't forget the sign change!</p></div>")

1,0,0,0
-1,1,0,0
-2,-3,1,0
-3,-4,0,1


Using the first column from E_1 and the 2nd column form E2, we see L =


In [33]:
LAcodes.title( "solving for y", sz=15)
y = L \ b
println("Use back-substitution to solve L y = b for y= $y"); println()
println("Compare the right hand side we got in the first (now unnecessary computation)")
println("\nFinish the computation using backsubstitution for the row echelon form problem U x = y")

HTML{String}("<div><p style=\"color:blue;font-size:15pt;height:15px;text-align:left;\">solving for y</p></div>")

Use back-substitution to solve L y = b for y= [-1.0, 0.0, -4.0, 2.0]

Compare the right hand side we got in the first (now unnecessary computation)

Finish the computation using backsubstitution for the row echelon form problem U x = y


In [43]:
U = A2[:,1:size(A,2)]
LAcodes.title( "Check L U = A")
println("L U - A = ")
Int64.(L*U - A)

HTML{String}("<div><p style=\"color:blue;font-size:20pt;height:15px;text-align:left;\">Check L U = A</p></div>")

L U - A = 


4×5 Array{Int64,2}:
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0

# PLU decomposition

This leaves a question: what if we have to do row exchanges?

It turns out to be a simple extension of the previous result.
Let's look at the following example:

In [52]:
A  = [ 1    3    1    4   0;
      -2    5    5    6   2;
       1    3    1    5   1;
       2  -16   -9  -19  -4;
      -1  -36  -22  -45  -3 ]


p1 = (1,0); E1 = [ 1 0 0 0 0; 2 1 0 0 0; -1 0 1 0 0; -2 0  0 1 0; 1 0 0 0 1]; A1 = E1 * A
p2 = (2,1); E2 = [ 1 0 0 0 0; 0 1 0 0 0;  0 0 1 0 0;  0 2  0 1 0; 0 3 0 0 1]; A2 = E2 * A1
p3 = (4,2); E3 = [ 1 0 0 0 0; 0 1 0 0 0;  0 0 0 1 0;  0 0  1 0 0; 0 0 0 0 1]; A3 = E3 * A2
p4 = (4,3); E4 = [ 1 0 0 0 0; 0 1 0 0 0;  0 0 1 0 0;  0 0 0 1 0; 0 0 0 -1 1]; A4 = E4 * A3

LAcodes.ge_layout( A, [E1 A1; E2 A2; E3 A3; E4 A4], [p1,(p1,p2),(p1,p2,(3,2)),(p1,p2,(3,2),p4),(p1,p2,(3,2),p4,(5,4))] )

,,,,,1,3,1,4,0
,,,,,-2,5,5,6,2
,,,,,1,3,1,5,1
,,,,,2,-16,-9,-19,-4
,,,,,-1,-36,-22,-45,-3
1,0,0,0,0,1,3,1,4,0
2,1,0,0,0,0,11,7,14,2
-1,0,1,0,0,0,0,0,1,1
-2,0,0,1,0,0,-22,-11,-27,-4
1,0,0,0,1,0,-33,-21,-41,-3
1,0,0,0,0,1,3,1,4,0


If we solve $E_3 E_2 E_1 A = U \Leftrightarrow  A = E_1^{-1} E_2^{-1} E_3^{-1} E_4^{-1} U$ and look at the product of the inverse matrices,
we see that the pattern no longer holds!

In [56]:
LAcodes.title("The inverse matrix is no longer unit lower triangular!")
LAcodes.ge_layout( Int64.(inv(E1)*inv(E2)*inv(E3)*inv(E4)))

HTML{String}("<div><p style=\"color:blue;font-size:20pt;height:15px;text-align:left;\">The inverse matrix is no longer unit lower triangular!</p></div>")

1,0,0,0,0
-2,1,0,0,0
1,0,0,1,0
2,-2,1,0,0
-1,-3,0,1,1


If we investigate, we see that the problem arises with the row exchange matrix $E_3$:<br>
Let's leave it out of the multiplication, and look at $(E_1^{-1} E_2^{-1}) E_3^{-1} E_4^{-1}$:

We find
$$ \begin{pmatrix} 1&0&0&0&0 \\
 -2 &1  &0 &0 &0 \\
  1 & 0 &1 &0 &0 \\
  2 &-2 &0 &1 &0 \\
 -1 &-3 &0 &0 &1 \end{pmatrix} E_3^{-1}
\begin{pmatrix}1& 0& 0&  0&  0 \\
 0 & 1 & 0 & 0 & 0 \\
 0 & 0 & 1 & 0 & 0 \\
 0 & 0 & 0 & 1 & 0 \\
 0 & 0 & 0 & 1 & 1\end{pmatrix}
$$
The product of the first and third matrix would be unit lower triangular as before.

A moment's thought leads to the realization that permuting the rows of $A$ appropriately should result in a matrix that does not require
a row exchange.<br>Could we somehow pull the second matrix to the left?

Since $E_3^{-1}$ is a permutation matrix, let's change notation and call it $P = E_3^{-1}$.
Denote the matrix to it's left $\mathscr{E} = E_1^{-1} E_2^{-1}$.<br>
The question we are asking is
whether we could interchange $\mathscr{E} P$.

---
$$\mathscr{E} P = P \mathscr{\tilde{E}} \Leftrightarrow \mathscr{\tilde{E}} = P^{-1} \mathscr{E} P$$

Let's see:
$$
P^{-1} \mathscr{E} = \begin{pmatrix}
  1 &  0 & 0 & 0 & 0\\
 -2 &  1 & 0 & 0 & 0\\
  2 & -2 & 0 & 1 & 0\\
  1 &  0 & 1 & 0 & 0\\
 -1 & -3 & 0 & 0 & 1 \end{pmatrix}
$$
so $P$ does indeed interchange the corresponding rows, but in the process, it displaces the 1 entries on the diagonal!<br>
Multiplying this matrix with $P$ from the right fixes up the 1 entries:
$$
P^{-1} \mathscr{E} = \begin{pmatrix}
  1 &  0 & 0 & 0 & 0\\
 -2 &  1 & 0 & 0 & 0\\
  2 & -2 & 1 & 0 & 0\\
  1 &  0 & 0 & 1 & 0\\
 -1 & -3 & 0 & 0 & 1 \end{pmatrix}
$$
The end result: pulling the row exchange matrix to the left exchanges the row entries below the diagonal.<br>
This is the right form so that the product of the elimination matrices are unit lower triangular as before.

> We have achieved the decomposition $A = P L U$.

In the class notes, we verify that the required patterns in the matrix multiplications hold in general.

> Using the PLU decomposition to solve $A x = b \Leftrightarrow P L U x = b \Leftrightarrow \begin{align} P w = b \\ L y = w \\ U x = y \end{align}$
> * $P w = b$ is just a row exchange of the entries in $b$.
> * $L y = w$ is solved by forward substitution. Its solution yields the right hand side of the echelon form $A x = b \Leftrightarrow U x = y$
> * $U x = y& is solved by back substitution as before

In [58]:
LAcodes.title( "Solve A x = b using A = PLU")
println("Use GE to find A = P L U")
P           = inv(E3)
Pinv_Einv_P = inv(P)*inv(E1)*inv(E2)*P
L           = Pinv_Einv_P * inv(E4)
U           = A4
println("P =" ); Base.print_matrix(stdout, Int64.(P)); println()
println("L =" ); Base.print_matrix(stdout, Int64.(L)); println()
println("U =" ); Base.print_matrix(stdout, Int64.(U)); println()
println()
println(" Check A - P L U")
Int64.(A - P*L*U)

HTML{String}("<div><p style=\"color:blue;font-size:20pt;height:15px;text-align:left;\">Solve A x = b using A = PLU</p></div>")

Use GE to find A = P L U
P =
 1  0  0  0  0
 0  1  0  0  0
 0  0  0  1  0
 0  0  1  0  0
 0  0  0  0  1
L =
  1   0  0  0  0
 -2   1  0  0  0
  2  -2  1  0  0
  1   0  0  1  0
 -1  -3  0  1  1
U =
 1   3  1   4  0
 0  11  7  14  2
 0   0  3   1  0
 0   0  0   1  1
 0   0  0   0  2

 Check A - P L U


5×5 Array{Int64,2}:
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0

In [59]:
b =[ 12; 10; 13; -38; -117]
println( "Here is a b = $b^t")

w = inv(P)*b
println( "so        w = $(Int64.(w))")

y = L \ w
println( "Fwd Subst y = $(Int64.(y))")

x = U \ y
println( "Bwd Subst x = $(Int64.(x))")

println("\nCheck b - A x = $(Int64.(b-A*x))")

Here is a b = [12, 10, 13, -38, -117]^t
so        w = [12, 10, -38, 13, -117]
Fwd Subst y = [12, 34, 6, 1, -4]
Bwd Subst x = [2, -1, 1, 3, -2]

Check b - A x = [0, 0, 0, 0, 0]
